In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index


array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def __len__(self):
        return len(self.X)


class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim=32):
        super(Encoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )

    def forward(self, x):
        return self.fc(x)

batch_size = 512

x_test_adv_0_1 = np.array(x_test_adv_0_1)
y_label_0_1 = np.array(y_label_0_1)
train_dataset = SimpleDataset(x_test_adv_0_1, y_label_0_1)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

input_dim = x_test_adv_0_1.shape[1]
embedding_dim = 32
net = Encoder(input_dim, embedding_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)



In [6]:
num_epochs = 20

for epoch in range(num_epochs):
    net.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = net(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.4f}")

net.eval()
def get_embeddings(net, data_loader):
    all_embeddings = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            embeddings = net(X_batch)
            all_embeddings.append(embeddings.cpu())
            all_labels.append(y_batch)
    return torch.cat(all_embeddings, dim=0), torch.cat(all_labels, dim=0)

train_embeddings, train_labels = get_embeddings(net, train_loader)


Epoch 1/20, Training Loss: 0.2162
Epoch 2/20, Training Loss: 0.0821
Epoch 3/20, Training Loss: 0.0725
Epoch 4/20, Training Loss: 0.0682
Epoch 5/20, Training Loss: 0.0654
Epoch 6/20, Training Loss: 0.0636
Epoch 7/20, Training Loss: 0.0622
Epoch 8/20, Training Loss: 0.0611
Epoch 9/20, Training Loss: 0.0601
Epoch 10/20, Training Loss: 0.0592
Epoch 11/20, Training Loss: 0.0585
Epoch 12/20, Training Loss: 0.0579
Epoch 13/20, Training Loss: 0.0573
Epoch 14/20, Training Loss: 0.0568
Epoch 15/20, Training Loss: 0.0562
Epoch 16/20, Training Loss: 0.0558
Epoch 17/20, Training Loss: 0.0555
Epoch 18/20, Training Loss: 0.0551
Epoch 19/20, Training Loss: 0.0547
Epoch 20/20, Training Loss: 0.0545


In [7]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]



In [8]:
def predict_by_nearest_Euclidean(train_embeddings, train_labels, test_embeddings, device='cpu', batch_size=1024):
    train_embeddings = train_embeddings.to(device)
    test_embeddings = test_embeddings.to(device)
    train_labels = train_labels.to(device)

    preds = []

    for i in range(0, test_embeddings.size(0), batch_size):
        batch_test = test_embeddings[i:i+batch_size]  # (batch_size, D)

        train_norm = (train_embeddings ** 2).sum(1).reshape(1, -1)  # (1, N_train)
        test_norm = (batch_test ** 2).sum(1).reshape(-1, 1)          # (batch_test, 1)
        inner_product = torch.matmul(batch_test, train_embeddings.T)  # (batch_test, N_train)
        dists = test_norm + train_norm - 2 * inner_product

        nearest_idx = torch.argmin(dists, dim=1)  # (batch_size,)
        batch_preds = train_labels[nearest_idx]
        preds.append(batch_preds.cpu())

    preds = torch.cat(preds, dim=0)
    return preds.numpy()



In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    X = X.astype(np.float32)
    test_dataset = SimpleDataset(X, y)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
    test_embeddings, test_labels = get_embeddings(net, test_loader)
    y_pred = predict_by_nearest_Euclidean(train_embeddings, train_labels, test_embeddings, device=device, batch_size=1024)



    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 3 4 5 7] [    10      9      2      1 228742   9989]
(10, 42) (10,)
Save 1 to baseline_Def2.npy
(9, 42) (9,)
Save 2 to baseline_Def3.npy
(2, 42) (2,)
Save 3 to baseline_Def4.npy
(1, 42) (1,)
Save 4 to baseline_Def5.npy
(228742, 42) (228742,)
Save 5 to baseline_Def6.npy
(9989, 42) (9989,)
Save 7 to baseline_Def8.npy
Execution Time: 22.428990 seconds
(716259,)
[0 1 2 3 4 5 7 8 9] [269477     22 193677    137   5083 241892   1153      1   4817]
(269477, 42) (269477,)
Save 0 to BIM_Def1.npy
(22, 42) (22,)
Save 1 to BIM_Def2.npy
(193677, 42) (193677,)
Save 2 to BIM_Def3.npy
(137, 42) (137,)
Save 3 to BIM_Def4.npy
(5083, 42) (5083,)
Save 4 to BIM_Def5.npy
(241892, 42) (241892,)
Save 5 to BIM_Def6.npy
(1153, 42) (1153,)
Save 7 to BIM_Def8.npy
(1, 42) (1,)
Save 8 to BIM_Def9.npy
(4817, 42) (4817,)
Save 9 to BIM_Def10.npy
Execution Time: 60.139219 seconds
(716259,)
[0 1 2 3 4 5 7 8 9] [448493     36   8385      2   6220 245118   2742      4   5259]
(448493, 42) (448493,)
Save 0 t

In [10]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []


for attack_name in attack_names:
    each = []
    num = 0
    each.append(f"Recommendation")
    for i in range(1, 12):
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_{attack_name}_Def{i}.npy"
        try:
            y = np.load(y_filename)
            num+=y.shape[0]
            each.append(y.shape[0])
        except FileNotFoundError:
            each.append(0)
    each.append(num)
    all_attack_list.append(each)

import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Recommendation_Euclidean.csv")



In [11]:
def predict_by_nearest_cosine(train_embeddings, train_labels, test_embeddings, device='cpu', batch_size=1024):
    train_embeddings = train_embeddings.to(device)
    test_embeddings = test_embeddings.to(device)
    train_labels = train_labels.to(device)

    train_embeddings = train_embeddings / torch.norm(train_embeddings, dim=1, keepdim=True)
    test_embeddings = test_embeddings / torch.norm(test_embeddings, dim=1, keepdim=True)

    preds = []

    for i in range(0, test_embeddings.size(0), batch_size):
        batch_test = test_embeddings[i:i+batch_size]  # (batch_size, D)

        cosine_sim = torch.matmul(batch_test, train_embeddings.T)  # (batch_test, N_train)
        cosine_dist = 1 - cosine_sim

        nearest_idx = torch.argmin(cosine_dist, dim=1)  # (batch_size,)
        batch_preds = train_labels[nearest_idx]
        preds.append(batch_preds.cpu())

    preds = torch.cat(preds, dim=0)
    return preds.numpy()

In [12]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    X = X.astype(np.float32)
    test_dataset = SimpleDataset(X, y)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
    test_embeddings, test_labels = get_embeddings(net, test_loader)
    y_pred = predict_by_nearest_cosine(train_embeddings, train_labels, test_embeddings, device=device, batch_size=1024)



    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_cosine/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_cosine/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 3 4 5 7] [    10      7      2      1 227016  11717]
(10, 42) (10,)
Save 1 to baseline_Def2.npy
(7, 42) (7,)
Save 2 to baseline_Def3.npy
(2, 42) (2,)
Save 3 to baseline_Def4.npy
(1, 42) (1,)
Save 4 to baseline_Def5.npy
(227016, 42) (227016,)
Save 5 to baseline_Def6.npy
(11717, 42) (11717,)
Save 7 to baseline_Def8.npy
Execution Time: 9.577642 seconds
(716259,)
[0 1 2 3 4 5 7 8 9] [460238     23  17789      2     19 237095   1082      1     10]
(460238, 42) (460238,)
Save 0 to BIM_Def1.npy
(23, 42) (23,)
Save 1 to BIM_Def2.npy
(17789, 42) (17789,)
Save 2 to BIM_Def3.npy
(2, 42) (2,)
Save 3 to BIM_Def4.npy
(19, 42) (19,)
Save 4 to BIM_Def5.npy
(237095, 42) (237095,)
Save 5 to BIM_Def6.npy
(1082, 42) (1082,)
Save 7 to BIM_Def8.npy
(1, 42) (1,)
Save 8 to BIM_Def9.npy
(10, 42) (10,)
Save 9 to BIM_Def10.npy
Execution Time: 27.935630 seconds
(716259,)
[0 1 2 3 4 5 7 8 9] [473457     45   3372      1    229 238152    994      1      8]
(473457, 42) (473457,)
Save 0 to FGSM_Def1.n

In [13]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []


for attack_name in attack_names:
    each = []
    num = 0
    each.append(f"Recommendation")
    for i in range(1, 12):
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_cosine/x_test_adv_{attack_name}_Def{i}.npy"
        try:
            y = np.load(y_filename)
            num+=y.shape[0]
            each.append(y.shape[0])
        except FileNotFoundError:
            each.append(0)
    each.append(num)
    all_attack_list.append(each)

import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Recommendation_cosine.csv")
all_attack_list


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Recommendation,0,10,7,2,1,227016,0,11717,0,0,0,238753
1,Recommendation,460238,23,17789,2,19,237095,0,1082,1,10,0,716259
2,Recommendation,473457,45,3372,1,229,238152,0,994,1,8,0,716259
3,Recommendation,460238,23,17789,2,19,237095,0,1082,1,10,0,716259
4,Recommendation,158,0,6458,45,10233,688136,0,2845,143,8241,0,716259
5,Recommendation,256409,24,219122,8,62,228602,0,12018,3,11,0,716259
6,Recommendation,0,39,25,6,5,681033,0,35151,0,0,0,716259
7,Recommendation,69546,9,23441,36729,2998,63795,12,516366,2396,958,9,716259
8,Recommendation,408143,1,69385,27,14,237890,0,798,1,0,0,716259
9,Recommendation,467974,20,9810,0,56,237332,0,1023,1,43,0,716259


In [14]:
def predict_by_nearest_manhattan(train_embeddings, train_labels, test_embeddings, device='cpu', batch_size=2048):
    train_embeddings = train_embeddings.to(device)
    test_embeddings = test_embeddings.to(device)
    train_labels = train_labels.to(device)

    preds = []

    for i in range(0, test_embeddings.size(0), batch_size):
        batch_test = test_embeddings[i:i+batch_size]  # (batch_size, D)

        batch_size_actual = batch_test.size(0)
        dists = torch.zeros(batch_size_actual, train_embeddings.size(0), device=device)
        for j in range(0, train_embeddings.size(0), batch_size):
            batch_train = train_embeddings[j:j+batch_size]
            abs_diff = torch.abs(batch_test.unsqueeze(1) - batch_train.unsqueeze(0))
            dists[:, j:j+batch_size] = abs_diff.sum(dim=2)

        nearest_idx = torch.argmin(dists, dim=1)  # (batch_size,)
        batch_preds = train_labels[nearest_idx]
        preds.append(batch_preds.cpu())

    preds = torch.cat(preds, dim=0)
    return preds.numpy()

In [15]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    X = X.astype(np.float32)
    test_dataset = SimpleDataset(X, y)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
    test_embeddings, test_labels = get_embeddings(net, test_loader)
    y_pred = predict_by_nearest_manhattan(train_embeddings, train_labels, test_embeddings, device=device, batch_size=1024)



    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 3 4 5 7] [    10      7      2      1 231342   7391]
(10, 42) (10,)
Save 1 to baseline_Def2.npy
(7, 42) (7,)
Save 2 to baseline_Def3.npy
(2, 42) (2,)
Save 3 to baseline_Def4.npy
(1, 42) (1,)
Save 4 to baseline_Def5.npy
(231342, 42) (231342,)
Save 5 to baseline_Def6.npy
(7391, 42) (7391,)
Save 7 to baseline_Def8.npy
Execution Time: 443.946906 seconds
(716259,)
[0 1 2 3 4 5 7 8 9] [258893      6 199577    104  10800 189883  55838      5   1153]
(258893, 42) (258893,)
Save 0 to BIM_Def1.npy
(6, 42) (6,)
Save 1 to BIM_Def2.npy
(199577, 42) (199577,)
Save 2 to BIM_Def3.npy
(104, 42) (104,)
Save 3 to BIM_Def4.npy
(10800, 42) (10800,)
Save 4 to BIM_Def5.npy
(189883, 42) (189883,)
Save 5 to BIM_Def6.npy
(55838, 42) (55838,)
Save 7 to BIM_Def8.npy
(5, 42) (5,)
Save 8 to BIM_Def9.npy
(1153, 42) (1153,)
Save 9 to BIM_Def10.npy
Execution Time: 1328.915272 seconds
(716259,)
[0 1 2 3 4 5 7 8 9] [448708     21   7774      4   7501 246796   3163     21   2271]
(448708, 42) (448708,)
Sav

In [16]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []


for attack_name in attack_names:
    each = []
    num = 0
    each.append(f"Recommendation_manhattan")
    for i in range(1, 12):
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_{attack_name}_Def{i}.npy"
        try:
            y = np.load(y_filename)
            num+=y.shape[0]
            each.append(y.shape[0])
        except FileNotFoundError:
            each.append(0)
    each.append(num)
    all_attack_list.append(each)

import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Recommendation_manhattan.csv")
all_attack_list


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Recommendation_manhattan,0,10,7,2,1,231342,0,7391,0,0,0,238753
1,Recommendation_manhattan,258893,6,199577,104,10800,189883,0,55838,5,1153,0,716259
2,Recommendation_manhattan,448708,21,7774,4,7501,246796,0,3163,21,2271,0,716259
3,Recommendation_manhattan,258893,6,199577,104,10800,189883,0,55838,5,1153,0,716259
4,Recommendation_manhattan,138,0,5796,34,8853,688963,0,3095,109,9271,0,716259
5,Recommendation_manhattan,248075,9,96719,235,19651,328170,0,22272,12,1116,0,716259
6,Recommendation_manhattan,0,39,19,6,8,694014,0,22173,0,0,0,716259
7,Recommendation_manhattan,80454,10,37422,28161,3427,45896,0,515896,3130,1845,18,716259
8,Recommendation_manhattan,311770,22,144245,20,9614,247487,0,2632,4,465,0,716259
9,Recommendation_manhattan,384721,4,75099,2,1505,253649,0,1215,0,64,0,716259
